# Phase 1 Project

![TakeOff](./images/take_off.jpg)

## 1.  Importing and loading data sets, initial inspection

In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('Data\AviationData.csv', encoding='latin-1', low_memory=False)
states_df = pd.read_csv('Data\\USState_Codes.csv')

In [106]:
# code the state names and abbreviations into the main df
states_dict = {}
for index, row in states_df.iterrows():
    states_dict[row[1]] = row[0]

    
def state_lookup(abbrev):
    if abbrev in states_dict.keys():
        return states_dict[abbrev]
    else:
        return None

df['State.Name'] = df['Location']
df['State.Abbrev'] = df['Location'].map(lambda x: str(x)[-2:])
df['State.Name'] = df['State.Abbrev'].map(lambda x: state_lookup(x) if state_lookup(x) != None else x)



In [107]:
df.head(3)

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,State.Name,State.Abbrev
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN,Idaho,ID
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996,California,CA
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007,Virginia,VA


In [108]:
df['State.Name'].value_counts().tail()

08    1
4C    1
O6    1
y,    1
65    1
Name: State.Name, dtype: int64

In [109]:
# take a look at the value counts
for column in df.columns:
    print(f'Column Name: {column}')
    print(f'{df[column].value_counts()} \n')

Column Name: Event.Id
20001212X19172    3
20001214X45071    3
20020917X03016    2
20001211X15101    2
20001214X36042    2
                 ..
20040608X00753    1
20001214X42487    1
20080602X00759    1
20001213X28131    1
20001214X42536    1
Name: Event.Id, Length: 87951, dtype: int64 

Column Name: Investigation.Type
Accident    85015
Incident     3874
Name: Investigation.Type, dtype: int64 

Column Name: Accident.Number
DCA22LA135     2
ERA22FA318     2
DCA22LA201     2
ERA22LA379     2
DCA22WA204     2
              ..
CHI91LA038     1
LAX97LA317     1
ANC91LA040     1
LAX94LA050A    1
LAX00LA252     1
Name: Accident.Number, Length: 88863, dtype: int64 

Column Name: Event.Date
1982-05-16    25
1984-06-30    25
2000-07-08    25
1986-05-17    24
1983-06-05    24
              ..
2020-01-28     1
2015-12-27     1
2014-11-01     1
2006-04-03     1
1997-02-28     1
Name: Event.Date, Length: 14782, dtype: int64 

Column Name: Location
ANCHORAGE, AK              434
MIAMI, FL             

## 1. Data Cleaning

In [110]:
#Dropping these columns due to Not enough data, not enough relevance
drop_columns = ['Latitude',          
                'Longitude',         
                'Schedule',          
                'Air.carrier',       
                'Airport.Code',      
                'FAR.Description',   
                'Publication.Date',  
                'Report.Status',
                'Accident.Number',
                'Registration.Number'
               ]
df = df.drop(columns=drop_columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Event.Date              88889 non-null  object 
 3   Location                88837 non-null  object 
 4   Country                 88663 non-null  object 
 5   Airport.Name            52790 non-null  object 
 6   Injury.Severity         87889 non-null  object 
 7   Aircraft.damage         85695 non-null  object 
 8   Aircraft.Category       32287 non-null  object 
 9   Make                    88826 non-null  object 
 10  Model                   88797 non-null  object 
 11  Amateur.Built           88787 non-null  object 
 12  Number.of.Engines       82805 non-null  float64
 13  Engine.Type             81812 non-null  object 
 14  Purpose.of.flight       82697 non-null

In [111]:
#Take a look at amateur built values
df['Amateur.Built'].value_counts()


No     80312
Yes     8475
Name: Amateur.Built, dtype: int64

In [112]:
#Take a look at amateur built NaN
df['Amateur.Built'].isna().sum()

102

In [113]:
# Stakeholders will not want to purchase airplanes built by amateurs for liability reasons
# dropping 'Amateur.Built' == 'Yes' or NaN 
df = df[(df['Amateur.Built'] == 'No') & (df['Amateur.Built'].notna())]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80312 entries, 0 to 88888
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                80312 non-null  object 
 1   Investigation.Type      80312 non-null  object 
 2   Event.Date              80312 non-null  object 
 3   Location                80265 non-null  object 
 4   Country                 80092 non-null  object 
 5   Airport.Name            47400 non-null  object 
 6   Injury.Severity         79313 non-null  object 
 7   Aircraft.damage         77165 non-null  object 
 8   Aircraft.Category       28721 non-null  object 
 9   Make                    80266 non-null  object 
 10  Model                   80245 non-null  object 
 11  Amateur.Built           80312 non-null  object 
 12  Number.of.Engines       74606 non-null  float64
 13  Engine.Type             73648 non-null  object 
 14  Purpose.of.flight       74242 non-null

In [114]:
# Take a closer look at country
df['Country'].value_counts()


United States       73906
Brazil                367
Mexico                348
Canada                346
United Kingdom        327
                    ...  
Montserrat              1
Malampa                 1
Seychelles              1
San Juan Islands        1
Benin                   1
Name: Country, Length: 217, dtype: int64

In [115]:
# Vast majority of this data set is from US crashes
# we shouldn't assume the model fits outside of US, removing non-US
df = df[df['Country'] == 'United States']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73906 entries, 0 to 88888
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                73906 non-null  object 
 1   Investigation.Type      73906 non-null  object 
 2   Event.Date              73906 non-null  object 
 3   Location                73896 non-null  object 
 4   Country                 73906 non-null  object 
 5   Airport.Name            46285 non-null  object 
 6   Injury.Severity         73798 non-null  object 
 7   Aircraft.damage         71949 non-null  object 
 8   Aircraft.Category       24697 non-null  object 
 9   Make                    73897 non-null  object 
 10  Model                   73891 non-null  object 
 11  Amateur.Built           73906 non-null  object 
 12  Number.of.Engines       72237 non-null  float64
 13  Engine.Type             71156 non-null  object 
 14  Purpose.of.flight       71481 non-null

In [116]:
# A lot of missing data in these catagorical columns but may be relevant data,
# so recoding the NaN as 'DATA MISSING' for now
missing_data_dict = {'Airport.Name': 'DATA MISSING',
                     'Aircraft.Category': 'DATA MISSING',
                     'Broad.phase.of.flight': 'DATA MISSING'}
df = df.fillna(missing_data_dict)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73906 entries, 0 to 88888
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                73906 non-null  object 
 1   Investigation.Type      73906 non-null  object 
 2   Event.Date              73906 non-null  object 
 3   Location                73896 non-null  object 
 4   Country                 73906 non-null  object 
 5   Airport.Name            73906 non-null  object 
 6   Injury.Severity         73798 non-null  object 
 7   Aircraft.damage         71949 non-null  object 
 8   Aircraft.Category       73906 non-null  object 
 9   Make                    73897 non-null  object 
 10  Model                   73891 non-null  object 
 11  Amateur.Built           73906 non-null  object 
 12  Number.of.Engines       72237 non-null  float64
 13  Engine.Type             71156 non-null  object 
 14  Purpose.of.flight       71481 non-null

In [117]:
# Take a closer look at 'Aircraft.Catagory' due to relevance
df['Aircraft.Category'].value_counts()


DATA MISSING         49209
Airplane             21121
Helicopter            2593
Glider                 472
Balloon                227
Weight-Shift           139
Powered Parachute       82
Gyrocraft               31
Ultralight              13
WSFT                     9
Blimp                    4
Powered-Lift             3
Unknown                  2
Rocket                   1
Name: Aircraft.Category, dtype: int64

In [118]:
# Not much data outside of Airplane and Helicopter, and stakeholders want high volume
# for revenue, so dropping all other 'Aircraft.Category'
df = df[(df['Aircraft.Category'] == 'Airplane') | 
        (df['Aircraft.Category'] == 'Helicopter') | 
        (df['Aircraft.Category'] == 'DATA MISSING')]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72923 entries, 0 to 88888
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                72923 non-null  object 
 1   Investigation.Type      72923 non-null  object 
 2   Event.Date              72923 non-null  object 
 3   Location                72913 non-null  object 
 4   Country                 72923 non-null  object 
 5   Airport.Name            72923 non-null  object 
 6   Injury.Severity         72815 non-null  object 
 7   Aircraft.damage         71069 non-null  object 
 8   Aircraft.Category       72923 non-null  object 
 9   Make                    72914 non-null  object 
 10  Model                   72908 non-null  object 
 11  Amateur.Built           72923 non-null  object 
 12  Number.of.Engines       71400 non-null  float64
 13  Engine.Type             70741 non-null  object 
 14  Purpose.of.flight       70503 non-null

In [119]:
# Take a closer look at 'Broad.phase.of.flight' due to relevance
df['Broad.phase.of.flight'].value_counts()


DATA MISSING    17542
Landing         14367
Takeoff         10919
Cruise           9164
Maneuvering      7090
Approach         5762
Taxi             1858
Climb            1798
Descent          1726
Go-around        1268
Standing          902
Unknown           428
Other              99
Name: Broad.phase.of.flight, dtype: int64

In [120]:
# 'Broad.phase.of.flight' catagories look ok
df['Broad.phase.of.flight'].value_counts()

DATA MISSING    17542
Landing         14367
Takeoff         10919
Cruise           9164
Maneuvering      7090
Approach         5762
Taxi             1858
Climb            1798
Descent          1726
Go-around        1268
Standing          902
Unknown           428
Other              99
Name: Broad.phase.of.flight, dtype: int64

In [121]:
# check out the data chronologically
df['Event.Date'] = pd.to_datetime(df['Event.Date'])
df['Event.Date'].dt.year.value_counts()


1982    3245
1983    3236
1984    3190
1985    2882
1986    2644
1987    2617
1988    2494
1989    2323
1990    2278
1991    2219
1992    2052
1993    2041
1995    1993
1994    1981
1996    1900
1999    1845
1997    1839
1998    1830
2000    1798
2003    1710
2001    1676
2002    1646
2005    1589
2007    1555
2004    1551
2008    1422
2006    1416
2011    1328
2010    1300
2009    1286
2012    1282
2016    1123
2013    1116
2018    1102
2017    1101
2019    1087
2015    1084
2022    1083
2014    1070
2021    1034
2020     948
1979       2
1977       1
1948       1
1981       1
1962       1
1974       1
Name: Event.Date, dtype: int64

In [124]:
#drop the very old one-offs, questionable data
df = df[df['Event.Date'].dt.year > 1981]

In [125]:
df['Event.Month'] = df['Event.Date'].dt.month
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72916 entries, 7 to 88888
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                72916 non-null  object        
 1   Investigation.Type      72916 non-null  object        
 2   Event.Date              72916 non-null  datetime64[ns]
 3   Location                72906 non-null  object        
 4   Country                 72916 non-null  object        
 5   Airport.Name            72916 non-null  object        
 6   Injury.Severity         72808 non-null  object        
 7   Aircraft.damage         71062 non-null  object        
 8   Aircraft.Category       72916 non-null  object        
 9   Make                    72907 non-null  object        
 10  Model                   72901 non-null  object        
 11  Amateur.Built           72916 non-null  object        
 12  Number.of.Engines       71394 non-null  float6

In [126]:
# Get a better idea of how much is missing now
df.isna().sum()
df.groupby('Engine.Type').count()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72916 entries, 7 to 88888
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                72916 non-null  object        
 1   Investigation.Type      72916 non-null  object        
 2   Event.Date              72916 non-null  datetime64[ns]
 3   Location                72906 non-null  object        
 4   Country                 72916 non-null  object        
 5   Airport.Name            72916 non-null  object        
 6   Injury.Severity         72808 non-null  object        
 7   Aircraft.damage         71062 non-null  object        
 8   Aircraft.Category       72916 non-null  object        
 9   Make                    72907 non-null  object        
 10  Model                   72901 non-null  object        
 11  Amateur.Built           72916 non-null  object        
 12  Number.of.Engines       71394 non-null  float6

In [127]:
df['Make'] = df['Make'].str.upper()
df['Model'] = df['Model'].str.replace(' ', '')

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72916 entries, 7 to 88888
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                72916 non-null  object        
 1   Investigation.Type      72916 non-null  object        
 2   Event.Date              72916 non-null  datetime64[ns]
 3   Location                72906 non-null  object        
 4   Country                 72916 non-null  object        
 5   Airport.Name            72916 non-null  object        
 6   Injury.Severity         72808 non-null  object        
 7   Aircraft.damage         71062 non-null  object        
 8   Aircraft.Category       72916 non-null  object        
 9   Make                    72907 non-null  object        
 10  Model                   72901 non-null  object        
 11  Amateur.Built           72916 non-null  object        
 12  Number.of.Engines       71394 non-null  float6

In [129]:
#Save out the reduced file for tableau visualization
df.to_csv('Data\AviationDataReduced.csv')